# Helping people with a lottery addiction

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018 .

The scenario we're following throughout this project is fictional — the main purpose is to practice applying the concepts we learned in a setting that simulates a real-world scenario.

In [1]:
import numpy as np
import pandas as pd

# Load code formatter
#%load_ext nb_black

## Defining a core functions

Throughout the project, we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

* A function that calculates factorials; and
* A function that calculates combinations.

In [2]:
# Define a function for factorial
def factorial(n):
    """Takes as input a number n and computes the factorial of that number n."""
    final_product = 1
    for i in range(n, 0, -1):
        final_product *= i
    return final_product


# Define a function for combinations
def combinations(n, k):
    """Returns the number of combinations when we're taking only k objects from a group of n objects."""
    return factorial(n) / (factorial(k) * factorial(n - k))

## Calculating one-ticket winning probability

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

The function should fulfill the following requirements:

* Take the list of 6 elements.
* Each of the elements should not bbe bigger than 49.
* Calculate and return theone-ticket winning probability.

In [3]:
def one_ticket_probability(user_list):
    """Return the probability of winning the 6/49 lotto using 1 ticket"""
    prob = 1 / combinations(49, 6)  # 6/49 lotto
    for element in user_list:
        if element > 49:
            return "All the values should be lower than 49. Please redefine your input"
    if len(user_list) != 6:
        return "Please enter 6 numbers. Not bigger, not less"
    return f"The probability of your win is {prob*100:.5f} %. It means you can win once in {int(combinations(49, 6))} attempts "

In [4]:
# Check the function
trials = [[1, 2, 3, 4, 5, 23], [3, 6, 9], [45, 21, 57, 19, 14, 22]]
for trial in trials:
    print(trial, one_ticket_probability(trial))

[1, 2, 3, 4, 5, 23] The probability of your win is 0.00001 %. It means you can win once in 13983816 attempts 
[3, 6, 9] Please enter 6 numbers. Not bigger, not less
[45, 21, 57, 19, 14, 22] All the values should be lower than 49. Please redefine your input


## Historical data check for Canada lottery

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

In [5]:
lottery = pd.read_csv("649.csv")
lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [6]:
lottery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT          3665 non-null   int64 
 1   DRAW NUMBER      3665 non-null   int64 
 2   SEQUENCE NUMBER  3665 non-null   int64 
 3   DRAW DATE        3665 non-null   object
 4   NUMBER DRAWN 1   3665 non-null   int64 
 5   NUMBER DRAWN 2   3665 non-null   int64 
 6   NUMBER DRAWN 3   3665 non-null   int64 
 7   NUMBER DRAWN 4   3665 non-null   int64 
 8   NUMBER DRAWN 5   3665 non-null   int64 
 9   NUMBER DRAWN 6   3665 non-null   int64 
 10  BONUS NUMBER     3665 non-null   int64 
dtypes: int64(10), object(1)
memory usage: 315.1+ KB


The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn

In [7]:
def extract_numbers(row):
    """Extracting winning number from a particular row of the dataset"""
    row = row[4:10]
    return set(row.values)

In [8]:
winning_numbers = lottery.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [9]:
def check_historical_occurence(user_list, winning_numbers):
    """Return the information about the number of times the combination inputted by the user occurred in the past."""
    user_set = set(user_list)
    occurences = 0
    for winning_set in winning_numbers:
        if user_set == winning_set:
            occurences += 1
    return (
        f"Your combination occured {occurences} time[s] in the last 36 years.",
        one_ticket_probability(user_list),
    )

In [10]:
# Check the function
trials_hist = [[1, 2, 3, 4, 5, 23], [3, 41, 11, 12, 43, 14], [45, 21, 57, 19, 14, 22]]
for i, trial in enumerate(trials_hist):
    print(
        f"trial {i}:",
        "\n",
        trial,
        "\n",
        check_historical_occurence(trial, winning_numbers),
        "\n",
    )

trial 0: 
 [1, 2, 3, 4, 5, 23] 
 ('Your combination occured 0 time[s] in the last 36 years.', 'The probability of your win is 0.00001 %. It means you can win once in 13983816 attempts ') 

trial 1: 
 [3, 41, 11, 12, 43, 14] 
 ('Your combination occured 1 time[s] in the last 36 years.', 'The probability of your win is 0.00001 %. It means you can win once in 13983816 attempts ') 

trial 2: 
 [45, 21, 57, 19, 14, 22] 
 ('Your combination occured 0 time[s] in the last 36 years.', 'All the values should be lower than 49. Please redefine your input') 



## Calculating multi-ticket winning probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning

In [11]:
def multi_ticket_probability(tickets):
    """Returns probability of winning the jackpot given the number of tickets bought."""
    prob = tickets / combinations(49, 6)
    if tickets > combinations(49, 6):
        return "You can't buy more tickets that are available."
    if prob < 0.01:
        return f"The probability of your win is {prob*100:.5f} %. It means you can win once in {round(combinations(49, 6)/tickets)} attempts "
    elif prob >= 0.01:
        return f"The probability of your win is {prob*100:.2f} %. Are you sure you going to spend so much money on lottery tickets?"

In [12]:
tickets = [1, 10, 100, 10000, 1000000, 6991908, 13983816, 100000000]
for n_tickets in tickets:
    print(
        f"If you buy {n_tickets} ticket[s]:",
        "\n",
        multi_ticket_probability(n_tickets),
        "\n",
    )

If you buy 1 ticket[s]: 
 The probability of your win is 0.00001 %. It means you can win once in 13983816 attempts  

If you buy 10 ticket[s]: 
 The probability of your win is 0.00007 %. It means you can win once in 1398382 attempts  

If you buy 100 ticket[s]: 
 The probability of your win is 0.00072 %. It means you can win once in 139838 attempts  

If you buy 10000 ticket[s]: 
 The probability of your win is 0.07151 %. It means you can win once in 1398 attempts  

If you buy 1000000 ticket[s]: 
 The probability of your win is 7.15 %. Are you sure you going to spend so much money on lottery tickets? 

If you buy 6991908 ticket[s]: 
 The probability of your win is 50.00 %. Are you sure you going to spend so much money on lottery tickets? 

If you buy 13983816 ticket[s]: 
 The probability of your win is 100.00 %. Are you sure you going to spend so much money on lottery tickets? 

If you buy 100000000 ticket[s]: 
 You can't buy more tickets that are available. 



## Calculating probability for 2-5 winning numbers in one ticket

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers. We're only interested in outcomes that match **exactly** five numbers, not **at least** five numbers.

In [13]:
def probability_less_6(n_winning_numbers):
    """Takes in an integer between 2 and 5 and prints
    information about the chances of winning depending on the value of that integer."""
    comb_numb = combinations(6, n_winning_numbers)  # how many possible combinations with given quanity of winning numbers out of 6-number ticket exists
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)  # Number of succesful lottery outcomes for the each combination above
    total_outcomes = comb_numb * n_combinations_remaining
    
    prob = total_outcomes / combinations(49, 6)
    return f"The probability of having {n_winning_numbers} winning numbers out of 6 is {prob*100:.3f} %. It means you win once in {round(combinations(49, 6)/total_outcomes)} times"

In [14]:
winning_numbers = [2, 3, 4, 5]
for number in winning_numbers:
    print(probability_less_6(number))

The probability of having 2 winning numbers out of 6 is 13.238 %. It means you win once in 8 times
The probability of having 3 winning numbers out of 6 is 1.765 %. It means you win once in 57 times
The probability of having 4 winning numbers out of 6 is 0.097 %. It means you win once in 1032 times
The probability of having 5 winning numbers out of 6 is 0.002 %. It means you win once in 54201 times


## Conclusions

To help people dealing with lottery addiction the folllowing functions have been defined during this project:

* `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket.
* `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set.
* `multi_ticket_probability()` — calculates the winning probability for any number of bought tickets between 1 and 13,983,816.
* `probability_less_6()` — calculates the probability of having two, three, four or five winning numbers.